In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import requests
import pymongo
from config import api_key

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.wwf_species
collection = db.species

In [4]:
#Creating a list of endangered species by scraping 2 pages of https://www.worldwildlife.org/species/directory?page=1

endangered_animals = []
for i in range(1,3):

   url = "https://www.worldwildlife.org/species/directory?page=" + str(i)
   response = requests.get(url)
   time.sleep(1)

# Scrape page into Soup

   soup = bs(response.text, 'html')

   results = soup.find_all('td', class_ ='keep')

   for result in results:
      animal = result.text
      endangered_animals.append(animal)
print(endangered_animals)

['African Elephant', 'African Wild Dog', 'Albacore Tuna', 'Amazon River Dolphin', 'Amur Leopard', 'Amur Tiger', 'Arctic Fox', 'Arctic Wolf', 'Asian Elephant', 'Beluga', 'Bengal Tiger', 'Bigeye Tuna', 'Black Rhino', 'Black Spider Monkey', 'Black-footed Ferret', 'Blue Whale', 'Bluefin Tuna', 'Bonobo', 'Bornean Orangutan', 'Borneo Pygmy Elephant', 'Bowhead Whale', 'Brown Bear', 'Chimpanzee', 'Common Bottlenose Dolphin', 'Cross River Gorilla', 'Dolphins and Porpoises', 'Dugong', 'Eastern Lowland Gorilla', 'Elephant', 'Fin Whale', 'Forest Elephant', 'Galápagos Penguin', 'Ganges River Dolphin', 'Giant Panda', 'Giant Tortoise', 'Gorilla', 'Gray Whale', 'Great White Shark', 'Greater One-Horned Rhino', 'Greater Sage-Grouse', 'Green Turtle', 'Hawksbill Turtle', "Hector's Dolphin", 'Hippopotamus', 'Humphead Wrasse', 'Indian Elephant', 'Indochinese Tiger', 'Indus River Dolphin', 'Irrawaddy Dolphin', 'Jaguar', 'Javan Rhino', 'Leatherback Turtle', 'Loggerhead Turtle', 'Macaw', 'Malayan Tiger', 'Mari

In [5]:
# Going to each species web page and scraping for image, and other information pertaining to status, population,
# scientific name, description and image
base_url = "https://www.worldwildlife.org/species"
for animal in endangered_animals:
    # Error handling
    try:
# Getting to each species URL    
        animal_url = animal.replace(" ","-") 
        url = base_url +"/"+ animal_url
        response = requests.get(url)
# Scraping for information for each species        
        soup = bs(response.text, 'html')
        overview_species = soup.find('div', id='overview')
        all_overview = overview_species.find_all('div', class_='container')
        species_status = all_overview[0].text
        species_population = all_overview[1].text
        species_scientific_name = all_overview[2].text
        species_description = soup.find('div', class_='wysiwyg lead').text
        image_path = soup.find('img',class_='centered')
        # Run only if title, price, and link are available
        if (all_overview and species_description and image_path):
            print(image_path)
            species_image = image_path['src']
        
# Creating a dictionary and putting it all the scraped information together        
            species_post = {
                'Species Name' : animal,
                'Status' : species_status,
                'Population' : species_population,
                'Scientific Name' : species_scientific_name,
                'Species Description' : species_description,
                'Species Image URL' : species_image
            }
# Inserting the record in the database collection    
            collection.insert_one(species_post)
    except Exception as e:
        print(e)

<img alt="African elephant browsing on bush" class="centered" src="https://c402277.ssl.cf1.rackcdn.com/photos/882/images/hero_small/African_Elephant_7.27.2012_hero_and_circle_HI_53941.jpg?1345532748"/>
<img alt="African wild dogs in Zamibia" class="centered" src="https://c402277.ssl.cf1.rackcdn.com/photos/10341/images/hero_small/wild_dogs-web.jpg?1447180370"/>
<img alt="tuna 102948248" class="centered" src="https://c402277.ssl.cf1.rackcdn.com/photos/1466/images/carousel_small/tuna_102948248.jpg?1345545852"/>
<img alt="Amazon River Dolphin (Pink Dolphin)" class="centered" src="https://c402277.ssl.cf1.rackcdn.com/photos/787/images/hero_small/Amazon_River_Dolphin_7.25.2012_Circle_and_Hero_XL_257657.jpg?1470325710"/>
<img alt="Amur Leopard" class="centered" src="https://c402277.ssl.cf1.rackcdn.com/photos/878/images/hero_small/amur-leopard_99144569.jpg?1345532564"/>
<img alt="Amur Tiger Hero and Circle" class="centered" src="https://c402277.ssl.cf1.rackcdn.com/photos/1584/images/hero_small/

<img alt="Responsible Forstry Biodiversity Loss" class="centered" src="https://c402277.ssl.cf1.rackcdn.com/photos/1679/images/hero_small/Responsible_Forestry_8.10.2012_Biodiversiry_Loss_HI_7474.jpg?1345600687"/>
<img alt="javan rhino" class="centered" src="https://c402277.ssl.cf1.rackcdn.com/photos/9496/images/hero_small/_H9A0249.jpg?1436885827"/>
<img alt="Leatherback turtle (Dermochelys coriacea) underwater. Kei Islands, Moluccas, Indonesia. 21 November 2009" class="centered" src="https://c402277.ssl.cf1.rackcdn.com/photos/317/images/hero_small/SCR_289416.jpg?1345568766"/>
<img alt="Loggerhead Turtle" class="centered" src="https://c402277.ssl.cf1.rackcdn.com/photos/297/images/hero_small/MID_110648loggerhead-hero.jpg?1345568357"/>
list index out of range
<img alt="Malayan Tiger " class="centered" src="https://c402277.ssl.cf1.rackcdn.com/photos/1641/images/hero_small/shutterstock_6107227.jpg?1345599723"/>
<img alt="Marine iguana on a sandy beach in the Galapagos Islands, Ecuador." clas

In [6]:
# Display items in MongoDB collection
specie_listings = db.species.find()

for specie in specie_listings:
    print(specie)


{'_id': ObjectId('5cd214c26ca91491745936af'), 'Species Name': 'African Elephant', 'Status': '\nVulnerable\n', 'Population': '\nApproximately 415,000 in the wild\n', 'Scientific Name': '\nLoxodonta africana\n', 'Species Description': '\nThe African elephant is the largest animal walking the Earth. Their herds wander through 37 countries in Africa. They are easily recognized by their trunk that is used for communication and handling objects. And their large ears allow them to radiate excess heat. Upper incisor teeth develop into tusks in African elephants and grow throughout their lifetime. There are two subspecies of African elephants—the Savanna (or bush) elephant and the Forest elephant. Savanna elephants are larger than forest elephants, and their tusks curve outwards. In addition to being smaller, forest elephants are darker and their tusks are straighter and point downward. There are also differences in the size and shape of the skull and skeleton between the two subspecies.\nFores

In [13]:

# Search for articles that mention animals

Urls = []
nytimesurl = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"
 

for animals in endangered_animals:

    query =  animals

 

    # Build query URL

 

    query_url = nytimesurl + 'q='+query+'&api-key='+api_key+'&sort=newest'

#   query_url

# Request articles

    articles = requests.get(query_url).json()



# The "response" property in articles contains the actual articles

# list comprehension.

    articles_list = [article for article in articles["response"]["docs"]]
    for article in articles_list:     
        Urls.append(article["web_url"])

 
# Dictionary to be inserted as a MongoDB document

articles_dict = {

    'endagered_animal': animals,

    'url': [Urls]

        }

collection.insert_one(articles_dict)

 

[{'web_url': 'https://www.nytimes.com/2019/05/07/briefing/president-trump-denver-met-gala.html', 'snippet': 'Here’s what you need to know at the end of the day.', 'lead_paragraph': '(Want to get this briefing by email? Here’s the sign-up.) ', 'abstract': 'Here’s what you need to know at the end of the day.', 'blog': {}, 'source': 'The New York Times', 'multimedia': [{'rank': 0, 'subtype': 'xlarge', 'caption': None, 'credit': None, 'type': 'image', 'url': 'images/2019/05/07/briefing/050719evening-briefing-promo/07evening-briefing-slide-C860-articleLarge.jpg', 'height': 441, 'width': 600, 'legacy': {'xlarge': 'images/2019/05/07/briefing/050719evening-briefing-promo/07evening-briefing-slide-C860-articleLarge.jpg', 'xlargewidth': 600, 'xlargeheight': 441}, 'subType': 'xlarge', 'crop_name': 'articleLarge'}, {'rank': 0, 'subtype': 'popup', 'caption': None, 'credit': None, 'type': 'image', 'url': 'images/2019/05/07/briefing/050719evening-briefing-promo/07evening-briefing-slide-C860-popup.jpg'

[{'web_url': 'https://www.nytimes.com/2019/04/05/books/review/poetry-picture-books.html', 'snippet': 'Some of the best illustrated children’s books are not stories — they’re poems that use language, form and rhythm to let kids reflect, imagine and think.', 'lead_paragraph': 'Countless picture books follow the same narrative structure, in which a character faces a challenge and then — at the end of approximately 500 words — overcomes that challenge, or doesn’t. We call this story. “Read me a story,” a child might beg, and so we do.', 'abstract': 'Some of the best illustrated children’s books are not stories — they’re poems that use language, form and rhythm to let kids reflect, imagine and think.', 'print_page': '22', 'blog': {}, 'source': 'The New York Times', 'multimedia': [{'rank': 0, 'subtype': 'xlarge', 'caption': None, 'credit': None, 'type': 'image', 'url': 'images/2019/04/07/books/review/07Snyder-01/07Snyder-01-articleLarge.jpg', 'height': 444, 'width': 600, 'legacy': {'xlarge':

[{'web_url': 'https://www.nytimes.com/2018/10/09/magazine/poke-long-island-fresh-fish.html', 'snippet': 'The Hawaiian dish might be the best way to eat fresh fish from Long Island.', 'lead_paragraph': 'Great food comes at a cost, and sometimes it’s no sleep and a lot of coffee, a long drive in the dark toward eastern Long Island and fish. There is unbelievable bounty there for those who know where to look and what to do with the information presented to them by current and sea, by the behavior of birds, by experience, by pure happenstance.', 'abstract': 'The Hawaiian dish might be the best way to eat fresh fish from Long Island.', 'print_page': '22', 'blog': {}, 'source': 'The New York Times', 'multimedia': [{'rank': 0, 'subtype': 'xlarge', 'caption': None, 'credit': None, 'type': 'image', 'url': 'images/2018/10/14/magazine/14mag-eat-image1/14mag-eat-image1-articleLarge-v2.jpg', 'height': 350, 'width': 600, 'legacy': {'xlarge': 'images/2018/10/14/magazine/14mag-eat-image1/14mag-eat-ima

[{'web_url': 'https://www.nytimes.com/2019/04/29/books/review/vanessa-woods-brian-hare-dogs.html', 'snippet': 'There are more than 70,000 dog books on Amazon, in every conceivable genre. What do they say about our relationship with these pets?', 'lead_paragraph': '“What would the world be like without music or rivers or the green and tender grass?” the poet Mary Oliver asks in her book “Dog Songs.” “What would this world be like without dogs?”', 'abstract': 'There are more than 70,000 dog books on Amazon, in every conceivable genre. What do they say about our relationship with these pets?', 'print_page': '19', 'blog': {}, 'source': 'The New York Times', 'multimedia': [{'rank': 0, 'subtype': 'xlarge', 'caption': None, 'credit': None, 'type': 'image', 'url': 'images/2019/05/05/books/review/05Hare_Woods/merlin_153683091_0dfc2982-19c7-4785-9b80-2b6ca3e5b223-articleLarge.jpg', 'height': 641, 'width': 600, 'legacy': {'xlarge': 'images/2019/05/05/books/review/05Hare_Woods/merlin_153683091_0df

[{'web_url': 'https://www.nytimes.com/2019/04/09/world/asia/chinas-voracious-appetite-for-timber-stokes-fury-in-russia-and-beyond.html', 'snippet': 'After sharply restricting logging in its own forests, China turned to imports, overwhelming even a country with abundant resources: Russia.', 'lead_paragraph': 'From the Altai Mountains to the Pacific Coast, logging is ravaging Russia’s vast forests, leaving behind swathes of scarred earth studded with dying stumps.', 'abstract': 'After sharply restricting logging in its own forests, China turned to imports, overwhelming even a country with abundant resources: Russia.', 'print_page': '4', 'blog': {}, 'source': 'The New York Times', 'multimedia': [{'rank': 0, 'subtype': 'xlarge', 'caption': None, 'credit': None, 'type': 'image', 'url': 'images/2019/02/01/world/00china-timber-1-promo/00china-timber-1-promo-articleLarge-v2.jpg', 'height': 400, 'width': 600, 'legacy': {'xlarge': 'images/2019/02/01/world/00china-timber-1-promo/00china-timber-1-

...

[{'web_url': 'https://www.nytimes.com/2019/03/28/climate/david-bernhardt-interior-secretary-senate.html', 'snippet': 'Lawmakers must decide whether Mr. Bernhardt, who has faced conflict-of-interest criticisms, is the right person to oversee some 500 million acres of public land as well as the nation’s coastal waters.', 'lead_paragraph': 'WASHINGTON — President Trump’s choice to lead the Interior Department, David Bernhardt, a former oil lobbyist who has been accused of conflicts of interest, faced questioning Thursday from senators who must decide whether he is the right person to oversee some 500 million acres of public land and vast coastal waters.', 'abstract': 'Lawmakers must decide whether Mr. Bernhardt, who has faced conflict-of-interest criticisms, is the right person to oversee some 500 million acres of public land as well as the nation’s coastal waters.', 'print_page': '16', 'blog': {}, 'source': 'The New York Times', 'multimedia': [{'rank': 0, 'subtype': 'xlarge', 'caption': 

[{'web_url': 'https://www.nytimes.com/interactive/2019/04/10/t-magazine/adam-rapp-play.html', 'snippet': 'Adam Rapp’s original play “The Sand and the Snow,” written for T’s Culture issue.', 'lead_paragraph': 'Adam Rapp’s original play “The Sand and the Snow,” written for T’s Culture issue.', 'abstract': 'Adam Rapp’s original play “The Sand and the Snow,” written for T’s Culture issue.', 'print_page': '84', 'blog': {}, 'source': 'The New York Times', 'multimedia': [{'rank': 0, 'subtype': 'xlarge', 'caption': None, 'credit': None, 'type': 'image', 'url': 'images/2019/04/08/t-magazine/art/culture-extras-slide-M3CV/culture-extras-slide-M3CV-articleLarge.jpg', 'height': 744, 'width': 600, 'legacy': {'xlarge': 'images/2019/04/08/t-magazine/art/culture-extras-slide-M3CV/culture-extras-slide-M3CV-articleLarge.jpg', 'xlargewidth': 600, 'xlargeheight': 744}, 'subType': 'xlarge', 'crop_name': 'articleLarge'}, {'rank': 0, 'subtype': 'popup', 'caption': None, 'credit': None, 'type': 'image', 'url':

[{'web_url': 'https://www.nytimes.com/2019/05/07/briefing/president-trump-denver-met-gala.html', 'snippet': 'Here’s what you need to know at the end of the day.', 'lead_paragraph': '(Want to get this briefing by email? Here’s the sign-up.) ', 'abstract': 'Here’s what you need to know at the end of the day.', 'blog': {}, 'source': 'The New York Times', 'multimedia': [{'rank': 0, 'subtype': 'xlarge', 'caption': None, 'credit': None, 'type': 'image', 'url': 'images/2019/05/07/briefing/050719evening-briefing-promo/07evening-briefing-slide-C860-articleLarge.jpg', 'height': 441, 'width': 600, 'legacy': {'xlarge': 'images/2019/05/07/briefing/050719evening-briefing-promo/07evening-briefing-slide-C860-articleLarge.jpg', 'xlargewidth': 600, 'xlargeheight': 441}, 'subType': 'xlarge', 'crop_name': 'articleLarge'}, {'rank': 0, 'subtype': 'popup', 'caption': None, 'credit': None, 'type': 'image', 'url': 'images/2019/05/07/briefing/050719evening-briefing-promo/07evening-briefing-slide-C860-popup.jpg'

[{'web_url': 'https://www.nytimes.com/2019/05/02/us/politics/on-politics-biden-electability.html', 'snippet': 'In the On Politics newsletter: At Mr. Biden’s events in Iowa this week, voters said practicality, not passion, was the driving force in their support.', 'lead_paragraph': 'Hi. Welcome to On Politics, your guide to the day in national politics. I’m Lisa Lerer, your host.', 'abstract': 'In the On Politics newsletter: At Mr. Biden’s events in Iowa this week, voters said practicality, not passion, was the driving force in their support.', 'blog': {}, 'source': 'The New York Times', 'multimedia': [{'rank': 0, 'subtype': 'xlarge', 'caption': None, 'credit': None, 'type': 'image', 'url': 'images/2019/05/02/opinion/02onpolitics-pm-04/02onpolitics-pm-04-articleLarge.jpg', 'height': 450, 'width': 600, 'legacy': {'xlarge': 'images/2019/05/02/opinion/02onpolitics-pm-04/02onpolitics-pm-04-articleLarge.jpg', 'xlargewidth': 600, 'xlargeheight': 450}, 'subType': 'xlarge', 'crop_name': 'articl

KeyError: 'response'